In [ ]:
# !pip install pybamm   # run this on new machines without pybamm installed
import pybamm as pb

## Basic model
Consider a 1-dimensional SEI which extends from the surface of a planar negative electrode (cathode) at $x=0$ until $x=L$, where $L$ is the thickness of the SEI. Since the SEI is porous, some electrolyte will remain the region $0<x<L$, with a solvent concentration $c$. The solvent is transported via diffusion process modelled by:
$$ \frac{\partial c}{\partial t} = -\frac{\partial N}{\partial x}, \;\; N = -D(c)\frac{\partial c}{\partial x}$$

where:
- $t$ = time
- $N$ = solvent flux
- $D(c)$ = effective solvent diffusivity as a function of solvent conc.
- $x$ = distance from surface of cathode

At the electrode surface, the solvent is consumed during SEI growth reaction with the reaction flux $R$. Assuming fast diffusion of solvent (NOT diffusion limited) so that the conc. of solvent is fixed ($c= c_{\infty}$, bulk electrolyte conc.) at the SEI-electrolyte surface ($x=L$). Also assuming the conc. of solvent at the start is uniform and equal to bulk electrolyte conc., This gives the initial and boundary conditions of:
$$ \left. c \right \vert_{t=0}$$
$$ \left. N \right \vert_{x=0} = -R, \;\; \left. c \right \vert_{x=L} = c_{\infty} $$

As the SEI is growing, the boundary moves with time. The SEI thickness grows at a rate proportional to the reaction flux $R$, with a constant partial molar volume of the reaction product $\hat V$. Let the initial thickness be $L_0$, which gives:
$$ \frac{dL}{dt} = \hat VR, \left. L \right \vert_{t=0} = L_0 $$

Finally, assuming the SEI growth reaction is irreversible with a const. potential across the SEI, and the reaction flux is proportional to the conc. of solvent at the electrode-SEI surface ($x=0$). This gives the reaction flux $R$ as:
$$ \left. R = kc \right \vert_{x=0} $$

where $k$ = reaction constant (dependent on the potential across the SEI).

## Defining the moving boundary
The model proposed is a moving boundary problem, which needs to be fixed in order to be solved. A scaling is introduced:
$$ x = L \epsilon $$

Then, applying the chain rule:
$$ \frac{\partial}{\partial x} \rightarrow \frac{1}{L}\frac{\partial}{\partial \epsilon}, \;\; \frac{\partial}{\partial t} \rightarrow \frac{\partial}{\partial t} - \frac{L'(t)}{L(t)}\epsilon\frac{\partial}{\partial \epsilon}$$

which allows the original equation to be rewritten as:
$$ \frac{\partial c}{\partial t} = \frac{\hat VR}{L}\epsilon\frac{\partial c}{\partial \epsilon} + \frac{1}{L^2}\frac{\partial}{\partial \epsilon}\left (D(c)\frac{\partial c}{\partial \epsilon} \right) $$


## Building the model in PyBaMM
The main steps are:
1. Initialise model
2. Define parameters and variables
3. State governing equations
4. State boundary conditions
5. State initial conditions
6. State output variables

In [ ]:
# initialing and empty base model
model = pb.BaseModel()

# dimensional parameters
k = pb.Parameter("Reaction rate constant [m.s-1]")
L_0 = pb.Parameter("Initial thickness [m]")
V_hat = pb.Parameter("Partial molar volume [m3.mol-1]")
c_inf = pb.Parameter("Bulk electrolyte solvent concentration [mol.m-3]")

# diffusivity as a function of solvent concentration
def D(cc):
    return pb.FunctionParameter(
        "Diffusivity [m2.s-1]", {"Solvent concentration [mol.m-3]": cc}
    )


# define dimensionless parameters in the model
xi = pb.SpatialVariable("xi", domain="SEI layer", coord_sys="cartesian")
c = pb.Variable("Solvent concentration [mol.m-3]", domain="SEI layer")
L = pb.Variable("SEI thickness [m]")

In [ ]:
######################
# governing equations
######################
# SEI reaction flux
R = k * pb.BoundaryValue(c, "left")

# solvent concentration equation
N = -1 / L * D(c) * pb.grad(c)
dcdt = (V_hat * R) / L * pb.inner(xi, pb.grad(c)) - 1 / L * pb.div(N)

# SEI thickness equation
dLdt = V_hat * R

# add equations to dictionary
# keys = variables to be solved, values = RHS of governing equations for each variable
model.rhs = {c: dcdt, L: dLdt}

In [ ]:
# Boundary conditions
D_left = pb.BoundaryValue(
    D(c), "left"
)  # pb requires BoundaryValue(D(c)) and not D(BoundaryValue(c))
grad_c_left = R * L / D_left

c_right = c_inf

model.boundary_conditions = {
    c: {"left": (grad_c_left, "Neumann"), "right": (c_right, "Dirichlet")}
}

In [ ]:
# set initial conditions
c_init = c_inf
L_init = L_0
model.initial_conditions = {c: c_init, L: L_init}

In [ ]:
# output variables
model.variables = {
    "SEI thickness [m]": L,
    "SEI growth rate [m]": dLdt,
    "Solvent concentration [mol.m-3]": c,
}

In [ ]:
# define geometry
geometry = pb.Geometry(
    {"SEI layer": {xi: {"min": pb.Scalar(0), "max": pb.Scalar(1)}}}
)


def Diffusivity(cc):
    return cc * 10 ** (-12)


# parameter values (not physically based, for example only, refer to literature for better accuracy)
param = pb.ParameterValues(
    {
        "Reaction rate constant [m.s-1]": 1e-6,
        "Initial thickness [m]": 1e-6,
        "Partial molar volume [m3.mol-1]": 10,
        "Bulk electrolyte solvent concentration [mol.m-3]": 1,
        "Diffusivity [m2.s-1]": Diffusivity,
    }
)

# process model and geometry
param.process_model(model)
param.process_geometry(geometry)

# mesh and discretise
submesh_types = {"SEI layer": pb.Uniform1DSubMesh}
var_pts = {xi: 100}
mesh = pb.Mesh(geometry, submesh_types, var_pts)

spatial_methods = {"SEI layer": pb.FiniteVolume()}
disc = pb.Discretisation(mesh, spatial_methods)
disc.process_model(model)

In [ ]:
# solve
solver = pb.ScipySolver()
solution = solver.solve(model, [0, 3600]) # solve for 1hr

# post-process output variables
L_out = solution["SEI thickness [m]"]
c_out = solution["Solvent concentration [mol.m-3]"]

In [ ]:
# plotting the results
import matplotlib.pyplot as plt
import numpy as np

# plot SEI thickness in microns as a function of t in microseconds
# and concentration in mol/m3 as a function of x in microns
L_0_eval = param.evaluate(L_0)
xi = np.linspace(0, 1, 100)  # dimensionless space


def plot(t):
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.plot(solution.t, L_out(solution.t) * 1e6)
    ax1.plot(t, L_out(t) * 1e6, "r.")
    ax1.set_ylabel(r"SEI thickness [$\mu$m]")
    ax1.set_xlabel(r"t [s]")

    ax2.plot(xi * L_out(t) * 1e6, c_out(t, xi))
    ax2.set_ylim(0, 1.1)
    ax2.set_xlim(0, L_out(solution.t[-1]) * 1e6)
    ax2.set_ylabel("Solvent concentration [mol.m-3]")
    ax2.set_xlabel(r"x [$\mu$m]")

    plt.tight_layout()
    plt.show()


import ipywidgets as widgets

widgets.interact(
    plot, t=widgets.FloatSlider(min=0, max=solution.t[-1], step=0.1, value=0)
);

In [ ]:
#save solution as a separate file
path = "C:/Users/jwang/OneDrive - Imperial College London/4_PROJECT/pybamm code/training/"
solution.save(path + "sample SEI test.pkl")

# loading a saved solution
#loaded_solution = pb.load(path + "sample SEI test.pkl")